In [1]:
import pandas as pd
from PIL import Image
import os

In [2]:
# Load Excel file
df = pd.read_excel('mapping.xlsx')

In [3]:
df.head()

,ID,Brand,Image,Description
0,1,Gucci,gucci1.jpg,Inspired by the summer spirit and beach clubs ...
1,2,Gucci,gucci2.jpg,Inspired by the summer spirit and beach clubs ...
2,3,Gucci,gucci3.jpg,Inspired by the summer spirit and beach clubs ...
3,4,Gucci,gucci4.jpg,Inspired by the summer spirit and beach clubs ...


In [4]:
# Define the path to the images folder
img_folder = 'Img'

# Function to load an image given the image name
def load_image(img_name):
    img_path = os.path.join(img_folder, img_name)
    return Image.open(img_path)

In [5]:
# Example: Load the first image and description
first_image_name = df.loc[0, 'Image']
first_description = df.loc[0, 'Description']

first_image = load_image(first_image_name)
first_image.show()  # This will display the image
print("Description:", first_description)

Description: Inspired by the summer spirit and beach clubs of the Italian coast, this item is part of the Gucci Lido line. This shirt is enlivened by a delicate Gucci flower and insect print. The detachable contrasting pointed collar offers style versatility.

White cotton poplin
Buttons covered with the same fabric
Gucci flower and insect print
Detachable pointed collar with ribbon closure at the front
Sleeves
Button closure
Length: 83cm, based on a size 40 (IT)
Made in Italy
The product shown in the image is a size 40 (IT)
Fabric: 100% Cotton.


In [6]:
df.index

RangeIndex(start=0, stop=4, step=1)

In [7]:
for i in df.index:
    first_image_name = df.loc[0, 'Image']
    first_description = df.loc[0, 'Description']
    df['Img'] = load_image(first_image_name)
    

# BLIP MODEL

In [8]:
from datasets import load_dataset 

dataset = load_dataset("ybelkada/football-dataset", split="train")

In [9]:
from torch.utils.data import Dataset, DataLoader

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], text=item["text"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

In [10]:
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:339: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:339: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [11]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)

In [14]:
dataset[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=722x360>,
 'text': "Benzema after Real Mardid's win against PSG"}

In [12]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()

for epoch in range(50):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)
    
    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

Epoch: 0


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Loss: 12.965131759643555
Loss: 10.281275749206543
Loss: 10.164142608642578
Epoch: 1
Loss: 10.158946990966797
Loss: 10.155778884887695
Loss: 10.178914070129395
Epoch: 2
Loss: 10.096150398254395
Loss: 10.120429992675781
Loss: 10.139711380004883
Epoch: 3
Loss: 10.11806583404541
Loss: 10.04734992980957
Loss: 10.055829048156738
Epoch: 4
Loss: 10.068819999694824
Loss: 10.022600173950195
Loss: 10.021385192871094
Epoch: 5
Loss: 9.986654281616211
Loss: 10.01785659790039
Loss: 9.984800338745117
Epoch: 6
Loss: 9.957784652709961
Loss: 9.93096923828125
Loss: 9.826619148254395
Epoch: 7
Loss: 9.469511032104492
Loss: 9.484317779541016
Loss: 8.853726387023926
Epoch: 8
Loss: 8.684438705444336
Loss: 8.460343360900879
Loss: 8.22873592376709
Epoch: 9
Loss: 7.974978923797607
Loss: 7.69453763961792
Loss: 7.5254621505737305
Epoch: 10
Loss: 7.267932891845703
Loss: 7.061933517456055
Loss: 6.896913051605225
Epoch: 11
Loss: 6.682538986206055
Loss: 6.529636383056641
Loss: 6.369488716125488
Epoch: 12
Loss: 6.170175

KeyboardInterrupt: 